In [ ]:
! pip install webdriver-manager selenium pandas time


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # run headless if desired

# Set up the Chrome driver using webdriver_manager
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Dummy DOB accepted by the website (format dd/mm/yyyy)
default_dob = "01/01/2000"

# Build the list of registration numbers:
# Original range: 22seai01 to 22seai70 plus four additional ones.
regnos = [f"22seai{str(i).zfill(2)}" for i in range(1, 75)]
extra_regnos = ["22seai56","22sead55", "22sead26", "21seai31", "22seai91", "22seai92", "22seai93", "22seai94", "22seai95", "22seai96"]  # update as needed
regnos.extend(extra_regnos)

# Define the exam batches to iterate over: (value, display_text)
# exam_batches = [
#     ("116", "APRIL 2023"),
#     ("118", "AUGUST 2023"),
#     ("121", "MARCH - 2024"),
#     ("123", "AUGUST - 2024")
# ]

exam_batches = [
    ("116", "APRIL 2023"),
    ("118", "AUGUST 2023"),
    ("119", "NOVEMBER - 2023"),
    ("121", "MARCH - 2024"),
    ("123", "AUGUST - 2024"),
    ("124", "AUGUST - 2024"),
    ("124", "AUGUST  2024"),
    ("125", "MARCH/APRIL - 2025")

]

# <span id="MainContent_lblSem" class="form-control-plaintext font-weight-bold">E - 5th Semester  (MARCH/APRIL - 2025)</span>


# List to store each student's data
students_data = []

for reg_no in regnos:
    print(f"Processing: {reg_no}")
    
    driver.get("https://results.uomexam.com/")
    
    # Wait for the page to load
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.TAG_NAME, "body"))
    )
    
    try:
        # Locate the registration number and DOB input fields using provided IDs.
        reg_input = driver.find_element(By.ID, "MainContent_txtRollNo")
        dob_input = driver.find_element(By.ID, "MainContent_txtDoB")
        
        # Clear and enter the registration number and dummy DOB
        reg_input.clear()
        reg_input.send_keys(reg_no)
        dob_input.clear()
        dob_input.send_keys(default_dob)
        
        # Locate and click the submit button using the updated ID.
        submit_button = driver.find_element(By.ID, "MainContent_btnSubmit")
        submit_button.click()
        
        # Wait until the results page loads; adjust timeout as needed.
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "MainContent_lblResultSummary"))
        )
        
        # --- Scrape common student details ---
        try:
            name_element = driver.find_element(By.ID, "MainContent_lblStudentName")
            student_name = name_element.text.strip()
        except Exception as e:
            print(f"Name not found for {reg_no}: {e}")
            student_name = None
        
        try:
            sem_element = driver.find_element(By.ID, "MainContent_lblSem")
            student_sem = sem_element.text.strip()
        except Exception as e:
            print(f"Semester not found for {reg_no}: {e}")
            student_sem = None
        
        # Prepare a dictionary for this student's data.
        student_record = {
            "regno": reg_no,
            "name": student_name,
            "semester": student_sem
        }
        
        # --- For each exam batch, select the option and scrape GPA ---
        gpa_list = []
        for value, label in exam_batches:
            try:
                # Find the dropdown and create a Select object.
                dropdown = Select(driver.find_element(By.ID, "MainContent_ddlExamBatch"))
                # Select the option by its value.
                dropdown.select_by_value(value)
                # Wait briefly for the GPA to update (adjust as needed).
                time.sleep(2)
                
                # Scrape the GPA from the result summary element.
                gpa_element = driver.find_element(By.ID, "MainContent_lblResultSummary")
                gpa_text = gpa_element.text.strip()  # Expected format: "GPA : 9.227"
                try:
                    gpa_value = float(gpa_text.split(':')[-1].strip())
                except Exception as parse_error:
                    print(f"Could not parse GPA for {reg_no} in {label}: {parse_error}")
                    gpa_value = None
                # Save the GPA in the student record.
                student_record[f"{label} gpa"] = gpa_value
                if gpa_value is not None:
                    gpa_list.append(gpa_value)
            except Exception as ex:
                print(f"Error processing exam batch {label} for {reg_no}: {ex}")
                student_record[f"{label} gpa"] = None
        
        # Calculate average GPA over the batches (if any GPA was successfully scraped)
        if gpa_list:
            # avg_gpa = sum(gpa_list) / len(gpa_list)
            avg_gpa = sum(gpa_list) / 4
        else:
            avg_gpa = None
        student_record["avg gpa"] = avg_gpa
        
        # Append the record to our list
        students_data.append(student_record)
        
    except Exception as e:
        print(f"Error processing {reg_no}: {e}")
        # In case of a failure, log an empty record or with error details.
        students_data.append({
            "regno": reg_no,
            "name": None,
            "semester": None,
            **{f"{label} gpa": None for _, label in exam_batches},
            "avg gpa": None
        })

# Convert the collected data into a DataFrame and save to CSV.
df = pd.DataFrame(students_data)
df.to_csv("resultsallf.csv", index=False)
print("Results saved to results.csv")

driver.quit()




Processing: 22seai01
Could not parse GPA for 22seai01 in AUGUST 2023: could not convert string to float: ''
Could not parse GPA for 22seai01 in AUGUST - 2024: could not convert string to float: ''
Error processing exam batch AUGUST - 2024 for 22seai01: Message: Cannot locate option with value: 124; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception

Error processing exam batch AUGUST  2024 for 22seai01: Message: Cannot locate option with value: 124; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception

Error processing exam batch MARCH/APRIL - 2025 for 22seai01: Message: Cannot locate option with value: 125; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception

Processing: 22seai02
Could not parse GPA for 22seai0

In [ ]:
import pandas as pd

# Read the CSV file
df = pd.read_csv(r"C:\VSCODE PROJECTS_PERSNOLSAM\nwwww\resultsallf.csv")

# Sort by "avg gpa" in descending order
df_sorted = df.sort_values(by="avg gpa", ascending=False)

# Save the sorted DataFrame to a new CSV file
df_sorted.to_csv("results_sorted.csv", index=False)
print("Sorted CSV saved as results_sorted.csv")


In [1]:
%pwd

'c:\\VSCODE PROJECTS_PERSNOLSAM\\nwwww'

In [6]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Set up the Chrome driver using webdriver_manager
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Dummy DOB accepted by the website (format dd/mm/yyyy)
default_dob = "01/01/2000"

# Build registration number list
regnos = [f"22seai{str(i).zfill(2)}" for i in range(1, 75)]
extra_regnos = ["22seai56", "22sead55", "22sead26", "21seai31",
                "22seai91", "22seai92", "22seai93", "22seai94", "22seai95", "22seai96"]
regnos.extend(extra_regnos)

# Define exam batches (value, display text)
exam_batches = [
    ("116", "APRIL 2023"),
    ("118", "AUGUST 2023"),
    ("119", "NOVEMBER - 2023"),
    ("121", "MARCH - 2024"),
    ("123", "AUGUST - 2024"),
    ("124", "AUGUST 2024"),
    ("125", "MARCH/APRIL - 2025"),
    ("129", "MARCH/APRIL - 2025")
]

# List to store all student data
students_data = []

# Function to extract GPA from text
def extract_gpa(text):
    try:
        return float(text.split(":")[-1].strip())
    except Exception:
        return None

# Iterate over each registration number
for reg_no in regnos:
    print(f"Processing: {reg_no}")
    driver.get("https://results.uomexam.com/")

    try:
        # Wait for the input fields to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "MainContent_txtRollNo"))
        )

        # Input registration number and DOB
        reg_input = driver.find_element(By.ID, "MainContent_txtRollNo")
        dob_input = driver.find_element(By.ID, "MainContent_txtDoB")
        reg_input.clear()
        reg_input.send_keys(reg_no)
        dob_input.clear()
        dob_input.send_keys(default_dob)

        # Submit the form
        submit_button = driver.find_element(By.ID, "MainContent_btnSubmit")
        submit_button.click()

        # Wait for the result summary to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "MainContent_lblResultSummary"))
        )

        # Initialize student record
        student_record = {"regno": reg_no}

        # Extract student name
        try:
            student_name = driver.find_element(By.ID, "MainContent_lblStudentName").text.strip()
        except Exception:
            student_name = None
            print(f"Name not found for {reg_no}")
        student_record["name"] = student_name

        # Extract semester
        try:
            student_sem = driver.find_element(By.ID, "MainContent_lblSem").text.strip()
        except Exception:
            student_sem = None
            print(f"Semester not found for {reg_no}")
        student_record["semester"] = student_sem

        gpa_list = []

        # Check for the presence of the exam batch dropdown
        try:
            dropdown_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "MainContent_ddlExamBatch"))
            )
            dropdown = Select(dropdown_element)
            available_values = [opt.get_attribute("value") for opt in dropdown.options]

            for value, label in exam_batches:
                if value not in available_values:
                    student_record[f"{label} gpa"] = None
                    continue

                try:
                    # Re-locate the dropdown to avoid stale element reference
                    dropdown_element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.ID, "MainContent_ddlExamBatch"))
                    )
                    dropdown = Select(dropdown_element)
                    dropdown.select_by_value(value)

                    # Wait for the result summary to update
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.ID, "MainContent_lblResultSummary"))
                    )

                    gpa_text = driver.find_element(By.ID, "MainContent_lblResultSummary").text.strip()
                    gpa_value = extract_gpa(gpa_text)
                    student_record[f"{label} gpa"] = gpa_value
                    if gpa_value is not None:
                        gpa_list.append(gpa_value)
                except Exception as ex:
                    print(f"Error handling {label} for {reg_no}: {ex}")
                    student_record[f"{label} gpa"] = None
        except Exception:
            print(f"No dropdown for {reg_no}, attempting single GPA fetch.")
            try:
                gpa_text = driver.find_element(By.ID, "MainContent_lblResultSummary").text.strip()
                gpa_value = extract_gpa(gpa_text)
                student_record["Only batch gpa"] = gpa_value
                if gpa_value is not None:
                    gpa_list.append(gpa_value)
            except Exception as ex:
                print(f"GPA fetch failed without dropdown for {reg_no}: {ex}")
                student_record["Only batch gpa"] = None

        # Calculate average GPA
        student_record["avg gpa"] = sum(gpa_list) / len(gpa_list) if gpa_list else None

        # Append the student record to the data list
        students_data.append(student_record)

    except Exception as e:
        print(f"Failed to process {reg_no}: {e}")
        # Append a record with None values in case of failure
        failed_record = {"regno": reg_no, "name": None, "semester": None}
        for _, label in exam_batches:
            failed_record[f"{label} gpa"] = None
        failed_record["Only batch gpa"] = None
        failed_record["avg gpa"] = None
        students_data.append(failed_record)

# Save all data to CSV
df = pd.DataFrame(students_data)
df.to_csv("ML_results.csv", index=False)
print("✅ Results saved to ML_results.csv")

# Close the browser
driver.quit()


Processing: 22seai01
Processing: 22seai02
Processing: 22seai03
Processing: 22seai04
Processing: 22seai05
Processing: 22seai06
Processing: 22seai07
Processing: 22seai08
Processing: 22seai09
Processing: 22seai10
Processing: 22seai11
Processing: 22seai12
Processing: 22seai13
Processing: 22seai14
Processing: 22seai15
Processing: 22seai16
Processing: 22seai17
Processing: 22seai18
Processing: 22seai19
Processing: 22seai20
Processing: 22seai21
Processing: 22seai22
Processing: 22seai23
Processing: 22seai24
Processing: 22seai25
Processing: 22seai26
Processing: 22seai27
Processing: 22seai28
Processing: 22seai29
Processing: 22seai30
Processing: 22seai31
Processing: 22seai32
Processing: 22seai33
Processing: 22seai34
Processing: 22seai35
Processing: 22seai36
Processing: 22seai37
Processing: 22seai38
Processing: 22seai39
Processing: 22seai40
Processing: 22seai41
Processing: 22seai42
Processing: 22seai43
Processing: 22seai44
Processing: 22seai45
Processing: 22seai46
Processing: 22seai47
Processing: 2

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# Configure Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--disable-gpu")
chrome_options.add_argument("--no-sandbox")

# Set up the Chrome driver using webdriver_manager
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=chrome_options)

# Dummy DOB accepted by the website (format dd/mm/yyyy)
default_dob = "01/01/2000"

# Build registration number list
regnos = [f"22sead{str(i).zfill(2)}" for i in range(1, 85)]
# extra_regnos = ["22seai56", "22sead55", "22sead26", "21seai31",
#                 "22seai91", "22seai92", "22seai93", "22seai94", "22seai95", "22seai96"]

extra_regnos = []

regnos.extend(extra_regnos)

# Define exam batches (value, display text)
exam_batches = [
    ("116", "APRIL 2023"),
    ("118", "AUGUST 2023"),
    ("119", "NOVEMBER - 2023"),
    ("121", "MARCH - 2024"),
    ("123", "AUGUST - 2024"),
    ("124", "AUGUST 2024"),
    ("125", "MARCH/APRIL - 2025"),
    ("129", "MARCH/APRIL - 2025")
]

# List to store all student data
students_data = []

# Function to extract GPA from text
def extract_gpa(text):
    try:
        return float(text.split(":")[-1].strip())
    except Exception:
        return None

# Iterate over each registration number
for reg_no in regnos:
    print(f"Processing: {reg_no}")
    driver.get("https://results.uomexam.com/")

    try:
        # Wait for the input fields to be present
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "MainContent_txtRollNo"))
        )

        # Input registration number and DOB
        reg_input = driver.find_element(By.ID, "MainContent_txtRollNo")
        dob_input = driver.find_element(By.ID, "MainContent_txtDoB")
        reg_input.clear()
        reg_input.send_keys(reg_no)
        dob_input.clear()
        dob_input.send_keys(default_dob)

        # Submit the form
        submit_button = driver.find_element(By.ID, "MainContent_btnSubmit")
        submit_button.click()

        # Wait for the result summary to load
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "MainContent_lblResultSummary"))
        )

        # Initialize student record
        student_record = {"regno": reg_no}

        # Extract student name
        try:
            student_name = driver.find_element(By.ID, "MainContent_lblStudentName").text.strip()
        except Exception:
            student_name = None
            print(f"Name not found for {reg_no}")
        student_record["name"] = student_name

        # Extract semester
        try:
            student_sem = driver.find_element(By.ID, "MainContent_lblSem").text.strip()
        except Exception:
            student_sem = None
            print(f"Semester not found for {reg_no}")
        student_record["semester"] = student_sem

        gpa_list = []

        # Check for the presence of the exam batch dropdown
        try:
            dropdown_element = WebDriverWait(driver, 5).until(
                EC.presence_of_element_located((By.ID, "MainContent_ddlExamBatch"))
            )
            dropdown = Select(dropdown_element)
            available_values = [opt.get_attribute("value") for opt in dropdown.options]

            for value, label in exam_batches:
                if value not in available_values:
                    student_record[f"{label} gpa"] = None
                    continue

                try:
                    # Re-locate the dropdown to avoid stale element reference
                    dropdown_element = WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.ID, "MainContent_ddlExamBatch"))
                    )
                    dropdown = Select(dropdown_element)
                    dropdown.select_by_value(value)

                    # Wait for the result summary to update
                    WebDriverWait(driver, 5).until(
                        EC.presence_of_element_located((By.ID, "MainContent_lblResultSummary"))
                    )

                    gpa_text = driver.find_element(By.ID, "MainContent_lblResultSummary").text.strip()
                    gpa_value = extract_gpa(gpa_text)
                    student_record[f"{label} gpa"] = gpa_value
                    if gpa_value is not None:
                        gpa_list.append(gpa_value)
                except Exception as ex:
                    print(f"Error handling {label} for {reg_no}: {ex}")
                    student_record[f"{label} gpa"] = None
        except Exception:
            print(f"No dropdown for {reg_no}, attempting single GPA fetch.")
            try:
                gpa_text = driver.find_element(By.ID, "MainContent_lblResultSummary").text.strip()
                gpa_value = extract_gpa(gpa_text)
                student_record["Only batch gpa"] = gpa_value
                if gpa_value is not None:
                    gpa_list.append(gpa_value)
            except Exception as ex:
                print(f"GPA fetch failed without dropdown for {reg_no}: {ex}")
                student_record["Only batch gpa"] = None

        # Calculate average GPA
        student_record["avg gpa"] = sum(gpa_list) / len(gpa_list) if gpa_list else None

        # Append the student record to the data list
        students_data.append(student_record)

    except Exception as e:
        print(f"Failed to process {reg_no}: {e}")
        # Append a record with None values in case of failure
        failed_record = {"regno": reg_no, "name": None, "semester": None}
        for _, label in exam_batches:
            failed_record[f"{label} gpa"] = None
        failed_record["Only batch gpa"] = None
        failed_record["avg gpa"] = None
        students_data.append(failed_record)

# Save all data to CSV
df = pd.DataFrame(students_data)
df.to_csv("resultsallf.csv", index=False)
print("✅ Results saved to resultsallf.csv")

# Close the browser
driver.quit()


Processing: 22sead01
Processing: 22sead02
Processing: 22sead03
Processing: 22sead04
Processing: 22sead05
Processing: 22sead06
Processing: 22sead07
Processing: 22sead08
Processing: 22sead09
Processing: 22sead10
Processing: 22sead11
Failed to process 22sead11: Message: 
Stacktrace:
	GetHandleVerifier [0x0x603763+63299]
	GetHandleVerifier [0x0x6037a4+63364]
	(No symbol) [0x0x431113]
	(No symbol) [0x0x47987e]
	(No symbol) [0x0x479c1b]
	(No symbol) [0x0x4c2212]
	(No symbol) [0x0x49e5c4]
	(No symbol) [0x0x4bfa4a]
	(No symbol) [0x0x49e376]
	(No symbol) [0x0x46d6e0]
	(No symbol) [0x0x46e544]
	GetHandleVerifier [0x0x85e033+2531347]
	GetHandleVerifier [0x0x859332+2511634]
	GetHandleVerifier [0x0x629eda+220858]
	GetHandleVerifier [0x0x61a528+156936]
	GetHandleVerifier [0x0x620c5d+183357]
	GetHandleVerifier [0x0x60b6c8+95912]
	GetHandleVerifier [0x0x60b870+96336]
	GetHandleVerifier [0x0x5f664a+9770]
	BaseThreadInitThunk [0x0x75695d49+25]
	RtlInitializeExceptionChain [0x0x77c2d03b+107]
	RtlGetAppCo

In [7]:
import pandas as pd

# Read the CSV file
df = pd.read_csv(r"C:\VSCODE PROJECTS_PERSNOLSAM\nwwww\ML_results.csv")

# Sort by "avg gpa" in descending order
df_sorted = df.sort_values(by="avg gpa", ascending=False)

# Save the sorted DataFrame to a new CSV file
df_sorted.to_csv("ML_results_sorted.csv", index=False)
print("Sorted CSV saved as ML_results_sorted.csv")


Sorted CSV saved as ML_results_sorted.csv
